In [10]:
import numpy as np

def generate_random_lattice_basis(low=-10000, high=10001):
    while True:
        basis = np.random.randint(low, high, size=(10, 10))
        if np.linalg.matrix_rank(basis) == 10:
            return basis

#生成一个随机格基
basis = generate_random_lattice_basis()
print(basis)
# 原始格基中最短的向量
shortest_vector = min(basis, key=lambda v: np.linalg.norm(v))
print("原始格基中最短的向量：", shortest_vector)
print("最短向量的范数：", np.linalg.norm(shortest_vector))

[[ 8659  8372 -3818 -4547  2881  7170  3219 -9257  4902  7938]
 [-5334 -2258  6495  2469 -9402   384  8347  2810 -2875  2291]
 [-2745 -1085 -8338 -5388  8632  -210 -6134  9233  1376 -4719]
 [-9382 -7107  1848 -5746 -8269  7635  1646  2688 -4580   917]
 [ 6973 -9063  -701  8477  9691  4743  2333   429  -264  4011]
 [ 2923 -6232 -7013  6435  1901 -8872  9988 -4222 -9356 -9803]
 [  347 -1327 -7423 -9978  7226  2527 -5928 -3941  9408   129]
 [ 5236 -7128 -6716  6045  2459 -7860 -9847  9158 -4890 -1082]
 [ 8511  5244  6809  7162 -2621 -7747  9872  7822 -3114  4432]
 [-1889 -7129  3678  1008 -4863 -8342 -5320  6335 -9220   392]]
原始格基中最短的向量： [-5334 -2258  6495  2469 -9402   384  8347  2810 -2875  2291]
最短向量的范数： 16169.693287134422


In [11]:
def gram_schmidt(B):
    n = B.shape[0]
    B_star = np.zeros_like(B, dtype=np.float64)
    mu = np.zeros((n, n), dtype=np.float64)
    for i in range(n):
        B_star[i] = B[i]
        for j in range(i):
            mu[i, j] = np.dot(B[i], B_star[j]) / np.dot(B_star[j], B_star[j])
            B_star[i] -= mu[i, j] * B_star[j]
    return B_star, mu

$$\text{To find the shortest non-zero vector, we use the following proposition: }$$
$$\| b_1 \| \le 2^{(n-1)/2)}\lambda_1 (\mathcal{L})$$
$$\text{For example, in dimension 10, the inequality becomes: }$$
$$\| b_1 \| \leq 16\lambda_1 (\mathcal{L}) \leq 2^{4.5}\lambda_1 (\mathcal{L})$$
$$\text{i.e. } \frac{1}{16}\| b_1 \| \leq \lambda_1 (\mathcal{L}) $$


In [12]:
def lll_reduction(B, delta=0.75):
    n = B.shape[0]
    B = B.astype(np.int64)
    k = 1
    B_star, mu = gram_schmidt(B)
    while k < n:
        for j in range(k-1, -1, -1):
            c = int(round(mu[k, j]))
            if c != 0:
                B[k] -= c * B[j]
                B_star, mu = gram_schmidt(B)
        if np.dot(B_star[k], B_star[k]) >= (delta - mu[k, k-1]**2) * np.dot(B_star[k-1], B_star[k-1]):
            k += 1
        else:
            B[[k, k-1]] = B[[k-1, k]]
            B_star,mu = gram_schmidt(B)
            k = max(k-1, 1)
    return B

# 生成一个随机格基并进行LLL约减
basis = generate_random_lattice_basis()
print("原始二十维格基：\n", basis)
reduced_basis = lll_reduction(basis)
print("LLL约减后的二十维格基：\n", reduced_basis)

print("LLL规约后的第一个向量：", reduced_basis[0])
print("LLL规约后的第一个向量的范数：", np.linalg.norm(reduced_basis[0]))

原始二十维格基：
 [[ 2180 -7505  -249   579  2863  1463  2876  7185  7164  2813]
 [ 3723  3230  1660  1216 -5069  -357 -6399  7380  8084  3353]
 [-6660  5336 -5213 -8999  7701 -4260 -9905  7324 -1055 -2881]
 [-1026  9682  7319 -4455  7791  9265 -2048  5112   944  6626]
 [-8561 -7081  9496 -4373 -8502 -5325  9965  7643 -4541 -4312]
 [-4171  9626  6830  5790  4633  7325 -8145  -300  7272  1386]
 [-7321   831  1666   423 -6734 -6163 -9395 -5798  6096  -675]
 [-4765  7246  1396 -2393  -450 -1746 -4439  7285  1306 -8108]
 [-3404 -6375  9741  5356 -9585  2305 -1089 -8944    39 -9222]
 [-5562 -8842 -8968  3023 -1135 -2312 -9154  1207 -5806  9359]]
LLL约减后的二十维格基：
 [[  5745  -5886   3126   -457    231   5597  -3559   4273   -334     33]
 [  -550   -487  -3115  -2489  -1871   5832   -552   5602  -2717   4374]
 [ -3565  -1619  -3375   1036   2632  -4134   6435   2912   7498   2780]
 [  3723   3230   1660   1216  -5069   -357  -6399   7380   8084   3353]
 [ -4176    887   2155  -9822  -3576  -4223  -3298  